In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

769

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                         "Current Description":description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | komsomolskiy
Processing Record 2 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | pisco
Processing Record 5 of Set 1 | khovu-aksy
Processing Record 6 of Set 1 | albany
Processing Record 7 of Set 1 | tulsipur
Processing Record 8 of Set 1 | makakilo city
Processing Record 9 of Set 1 | cherskiy
Processing Record 10 of Set 1 | haines junction
Processing Record 11 of Set 1 | inzer
Processing Record 12 of Set 1 | dalbandin
Processing Record 13 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | canton
Processing Record 16 of Set 1 | hofn
Processing Record 17 of Set 1 | butaritari
Processing Record 18 of Set 1 | sahrak
City not found. Skipping...
Processing Record 19 of Set 1 | san patricio
Processing Record 20 of Set 1 | bambous virieux
Processing Record 21 

Processing Record 36 of Set 4 | akhmeta
Processing Record 37 of Set 4 | vadso
Processing Record 38 of Set 4 | muravlenko
Processing Record 39 of Set 4 | chuy
Processing Record 40 of Set 4 | klaksvik
Processing Record 41 of Set 4 | ginda
Processing Record 42 of Set 4 | leningradskiy
Processing Record 43 of Set 4 | lebu
Processing Record 44 of Set 4 | te anau
Processing Record 45 of Set 4 | marawi
Processing Record 46 of Set 4 | helong
Processing Record 47 of Set 4 | vaitupu
City not found. Skipping...
Processing Record 48 of Set 4 | patrocinio
Processing Record 49 of Set 4 | kresttsy
Processing Record 50 of Set 4 | cotonou
Processing Record 1 of Set 5 | horodyshche
Processing Record 2 of Set 5 | anage
Processing Record 3 of Set 5 | san luis
Processing Record 4 of Set 5 | utinga
Processing Record 5 of Set 5 | obo
Processing Record 6 of Set 5 | tautira
Processing Record 7 of Set 5 | sault sainte marie
Processing Record 8 of Set 5 | guanhaes
Processing Record 9 of Set 5 | mrirt
City not fo

Processing Record 30 of Set 8 | trairi
Processing Record 31 of Set 8 | alta floresta
Processing Record 32 of Set 8 | nuuk
Processing Record 33 of Set 8 | jumla
Processing Record 34 of Set 8 | qaqortoq
Processing Record 35 of Set 8 | vardo
Processing Record 36 of Set 8 | golden
Processing Record 37 of Set 8 | buba
Processing Record 38 of Set 8 | tessalit
Processing Record 39 of Set 8 | akyab
Processing Record 40 of Set 8 | opuwo
Processing Record 41 of Set 8 | yar-sale
Processing Record 42 of Set 8 | zeya
Processing Record 43 of Set 8 | puerto colombia
Processing Record 44 of Set 8 | samusu
City not found. Skipping...
Processing Record 45 of Set 8 | saryg-sep
Processing Record 46 of Set 8 | santa clarita
Processing Record 47 of Set 8 | bulgan
Processing Record 48 of Set 8 | kutum
Processing Record 49 of Set 8 | monrovia
Processing Record 50 of Set 8 | flinders
Processing Record 1 of Set 9 | lantawan
Processing Record 2 of Set 9 | palmer
Processing Record 3 of Set 9 | vana-vigala
Process

Processing Record 19 of Set 12 | komae
Processing Record 20 of Set 12 | honiara
Processing Record 21 of Set 12 | ippy
Processing Record 22 of Set 12 | saurimo
Processing Record 23 of Set 12 | ludvika
Processing Record 24 of Set 12 | aromashevo
Processing Record 25 of Set 12 | poso
Processing Record 26 of Set 12 | hun
Processing Record 27 of Set 12 | toumodi
Processing Record 28 of Set 12 | veraval
Processing Record 29 of Set 12 | penzance
Processing Record 30 of Set 12 | karratha
Processing Record 31 of Set 12 | guajara-mirim
City not found. Skipping...
Processing Record 32 of Set 12 | keokuk
Processing Record 33 of Set 12 | san pedro
Processing Record 34 of Set 12 | keetmanshoop
Processing Record 35 of Set 12 | berlevag
Processing Record 36 of Set 12 | haibowan
City not found. Skipping...
Processing Record 37 of Set 12 | faya
Processing Record 38 of Set 12 | belyy yar
Processing Record 39 of Set 12 | snezhnogorsk
Processing Record 40 of Set 12 | banda aceh
Processing Record 41 of Set 

Processing Record 2 of Set 16 | blackfoot
Processing Record 3 of Set 16 | safaga
Processing Record 4 of Set 16 | mildura
Processing Record 5 of Set 16 | nijar
Processing Record 6 of Set 16 | shache
Processing Record 7 of Set 16 | kathmandu
Processing Record 8 of Set 16 | sivaki
Processing Record 9 of Set 16 | murupara
Processing Record 10 of Set 16 | rawannawi
City not found. Skipping...
Processing Record 11 of Set 16 | katherine
Processing Record 12 of Set 16 | santa marta
Processing Record 13 of Set 16 | seymchan
Processing Record 14 of Set 16 | atasu
Processing Record 15 of Set 16 | carupano
Processing Record 16 of Set 16 | gunjur
Processing Record 17 of Set 16 | thunder bay
Processing Record 18 of Set 16 | myitkyina
Processing Record 19 of Set 16 | korgen
-----------------------------
Data Retrieval Complete      
-----------------------------


In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Komsomolskiy,40.4272,71.7189,44.02,33,0,2.30,UZ,2022-02-05 07:06:12,clear sky
1,Vaini,-21.2000,-175.2000,77.16,94,100,10.36,TO,2022-02-05 07:06:13,light rain
2,Pisco,-13.7000,-76.2167,66.25,82,0,4.61,PE,2022-02-05 07:06:13,clear sky
3,Khovu-Aksy,51.1333,93.6000,2.62,80,21,2.73,RU,2022-02-05 07:06:14,few clouds
4,Albany,42.6001,-73.9662,15.31,85,100,1.99,US,2022-02-05 07:04:03,overcast clouds
5,Tulsipur,28.1310,82.2973,61.05,57,27,5.75,NP,2022-02-05 07:06:14,scattered clouds
6,Makakilo City,21.3469,-158.0858,71.29,79,0,3.44,US,2022-02-05 07:06:15,clear sky
7,Cherskiy,68.7500,161.3000,-31.86,97,10,3.58,RU,2022-02-05 07:06:15,clear sky
8,Haines Junction,60.7522,-137.5108,4.44,99,100,2.55,CA,2022-02-05 07:06:16,overcast clouds
9,Inzer,54.2167,57.5556,28.65,78,100,4.18,RU,2022-02-05 07:06:16,overcast clouds


In [14]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Komsomolskiy,UZ,40.4272,71.7189,44.02,33,0,2.30,clear sky
1,Vaini,TO,-21.2000,-175.2000,77.16,94,100,10.36,light rain
2,Pisco,PE,-13.7000,-76.2167,66.25,82,0,4.61,clear sky
3,Khovu-Aksy,RU,51.1333,93.6000,2.62,80,21,2.73,few clouds
4,Albany,US,42.6001,-73.9662,15.31,85,100,1.99,overcast clouds


In [16]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")